In [1]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import DictConfig, OmegaConf
from transformers import AutoTokenizer

from data import PunctuationDataModule, PunctuationInferenceDataset
import os
from models import PunctuationDomainModel

from nemo.utils.exp_manager import exp_manager
from time import time
from pytorch_lightning.callbacks import ModelCheckpoint

import atexit
from copy import deepcopy
import snoop
snoop.install()

from hydra.experimental import initialize, initialize_config_module, initialize_config_dir, compose
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize()
cfg=compose(
    config_name="test_config.yaml", 
)
cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}
ids_to_labels = {_[0]:_[1] for _ in enumerate(cfg.model.punct_label_ids)}

model = PunctuationDomainModel.load_from_checkpoint(
    checkpoint_path="/home/nxingyu/project/Punctuation_with_Domain_discriminator/2021-02-09_14-05-14/checkpoints/Punctuation_with_Domain_discriminator-last.ckpt")

trainer = pl.Trainer(**cfg.trainer)

14:59:48.20 LOG:


shuffling <data.punctuation_dataset_multi.PunctuationDomainDataset object at 0x7f44ea05e5b0>


14:59:48.28 .... os.system('bash data/shuffle.sh -i {} -o {} -a {} -s {} -m {} -t {}'.format(self.target_file, self.target_file, ['true','false'][randomize], seed, '100M',self.tmp_path)) = 0
14:59:48.34 LOG:
14:59:48.63 .... f"1st {n} encoder layers of transformer frozen" = '1st 11 encoder layers of transformer frozen'
GPU available: True, used: False
TPU available: None, using: 0 TPU cores
[NeMo W 2021-02-09 14:59:48 nemo_logging:349] /home/nxingyu/miniconda3/envs/NLP/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
      warnings.warn(*args, **kwargs)
    


In [8]:
queries = [
    'we bought four shirts one pen and a mug from the nvidia gear store in santa clara',
    'what can i do for you today',
    'how are you',
    'good morning everyone how have your weekends been its a really great day'
]
inference_results = model.add_punctuation(queries)
inference_results

[' we bought four shirts, one pen, and a mug from the nvidia gear store in santa clara                                                                                                            ',
 ' what can i do for you today?                                                                                                                        ',
 ' , how are you? ,                                                                                                                           ',
 ' good morning. everyone? how have your weekends been? its a really great day?                                                                                                                  ']